# 07. 데이터베이스에 저장하기
- Item Pipeline 기능 사용

## 데이터베이스에 아이템 저장하기

### MySQL에 신규 데이터베이스 테이블 만들기
- 각각의 quote는 여러 개의 태그를 가지며, 각각의 태그도 여러 개의 격언을 가지며 이런 관계를 다 대 다(Many-to-Many) 관계라고 함

In [1]:
import MySQLdb

connection = MySQLdb.connect(
    user='root',
    passwd='3756',
    host='localhost'
)

cursor = connection.cursor()
sql = 'CREATE DATABASE quotes DEFAULT CHARACTER SET utf8'
cursor.execute(sql)

1

In [2]:
sql = 'use quotes'
cursor.execute(sql)

0

`` `text_hash` char(64) DEFAULT NULL``과 ``UNIQUE KEY `text_hash` (`text_hash`)`` : quote의 동일성을 확인하기 위한 칼럼과 인덱스로, 내용이 동일하면 같은 아이템으로 판단하고 저장하지 않음

In [5]:
sql = 'CREATE TABLE `quotes` ( \
    `id` int(11) unsigned NOT NULL AUTO_INCREMENT, \
    `author` varchar(255) DEFAULT NULL, \
    `text` text, \
    `text_hash` char(64) DEFAULT NULL, \
    `created_at` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP, \
    `updated_at` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP, \
    PRIMARY KEY (`id`), \
    UNIQUE KEY `text_hash` (`text_hash`) \
) ENGINE=InnoDB AUTO_INCREMENT=3 DEFAULT CHARSET=utf8mb4'

cursor.execute(sql)

0

In [6]:
sql = "CREATE TABLE `tags` ( \
    `id` int(11) unsigned NOT NULL AUTO_INCREMENT, \
    `name` varchar(255) NOT NULL DEFAULT '', \
    `created_at` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP, \
    `updated_at` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP, \
    PRIMARY KEY (`id`), \
    UNIQUE KEY `tag` (`name`) \
) ENGINE=InnoDB AUTO_INCREMENT=7 DEFAULT CHARSET=utf8mb4"

cursor.execute(sql)

0

In [7]:
sql = "CREATE TABLE `quotes_tags` ( \
    `id` int(11) unsigned NOT NULL AUTO_INCREMENT, \
    `quote_id` int(11) NOT NULL, \
    `tag_id` int(11) NOT NULL, \
    `created_at` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP, \
    `updated_at` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP, \
    PRIMARY KEY (`id`), \
    UNIQUE KEY `quote_tag` (`quote_id`, `tag_id`) \
) ENGINE=InnoDB AUTO_INCREMENT=7 DEFAULT CHARSET=utf8mb4"

cursor.execute(sql)

0

### pipeline 작성하기
- 데이터베이스 조작에는 Orator 사용
- `my_project/my_project/pipelines.py`

```python
from hashlib import sha256
from orator import DatabaseManager, Model
from orator.orm import belongs_to_many

from my_project.settings import ORATOR_CONFIG

db = DatabaseManager(ORATOR_CONFIG)
Model.set_connection_resolver(db)

class Quote(Model):
    """quotes 테이블 모델"""

    @belongs_to_many
    def tags(self):
        return Tag

class Tag(Model):
    """tags 테이블 모델"""

    @belongs_to_many
    def quote(self):
        return Quote

class DatabasePipeline(object):
    """MySQL에 Quotes 저장하기"""

    def __init__(self):
        """스크레이핑한 모든 item을 저장할 변수 선언"""
        self.items = []

    def process_item(self, item, spider):
        """각각의 아이템에 대한 처리"""
        self.items.append(item)
        return item

    def close_spider(self, spider):
        """spider 종료 후의 처리"""
        for item in self.items:
            text_hash = sha256(
                        item['text'].encode('utf8', 'ignore')).hexdigest()
            exist_quote = Quote.where('text_hash', text_hash).get()
            if exist_quote:
                continue
            quote = Quote()
            quote.author = item['author']
            quote.text = item['text']
            quote.text_hash = text_hash
            quote.save()

            tags = []
            for tag_name in item['tags']:
                tag = Tag.where('name', tag_name).first()
                if not tag:
                    tag = Tag()
                    tag.name = tag_name
                    tag.save()
                tags.append(tag)
                quote_tags = quote.tags()
                quote_tags.save(tag)
```

- `my_project/my_project/settings.py`
```python
(...생략...)
DEPTH_LIMIT = 1
ORATOR_CONFIG={
    'mysql':{
        'driver':'mysql',
        'host':'localhost',
        'database':'quotes',
        'user':'root',
        'password':'3756',
        'prefix':'',
        'log_queries':True,
    }
}
(...생략...)
ITEM_PIPELINES = {
    'my_project.pipelines.DatabasePipeline': 300,
}
(...생략...)
```

- `scrapy crawl quotes-1` 실행

In [13]:
import pandas as pd

sql = "select id, author, text from quotes"

cursor.execute(sql)
rows = cursor.fetchall()
pd.DataFrame(rows)

,0,1,2
0,1,Albert Einstein,“The world as we have created it is a process ...
1,2,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
2,3,Albert Einstein,“There are only two ways to live your life. On...
3,4,Jane Austen,"“The person, be it gentleman or lady, who has ..."
4,5,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."
5,6,Albert Einstein,“Try not to become a man of success. Rather be...
6,7,André Gide,“It is better to be hated for what you are tha...
7,8,Thomas A. Edison,"“I have not failed. I've just found 10,000 way..."
8,9,Eleanor Roosevelt,“A woman is like a tea bag; you never know how...
9,10,Steve Martin,"“A day without sunshine is like, you know, nig..."


In [14]:
sql = "select id, name from tags"

cursor.execute(sql)
rows = cursor.fetchall()
pd.DataFrame(rows)

,0,1
0,5,abilities
1,17,adulthood
2,12,aliteracy
3,16,be-yourself
4,13,books
5,1,change
6,6,choices
7,14,classic
8,2,deep-thoughts
9,21,edison


In [15]:
sql = "select id, quote_id, tag_id from quotes_tags"

cursor.execute(sql)
rows = cursor.fetchall()
pd.DataFrame(rows)

,0,1,2
0,1,1,1
1,2,1,2
2,3,1,3
3,4,1,4
4,5,2,5
5,6,2,6
6,7,3,7
7,8,3,8
8,9,3,9
9,10,3,10


In [ ]:
cursor.close()
connection.close()